In [104]:
import random
from threading import Thread, Event

In [105]:
class ActionsInterfaceList(list):
    def __init__(self,actions):
        super(ActionsInterfaceList,self).__init__()
        for i,action in enumerate(actions):
            self.append((i,action.public_interface))
    
class Action(object):
    def __init__(self):
        self.text = ""
    
    @property
    def public_interface(self):
        return [self.text,self.is_terminal]
        
    def _callback(self,*args,**kwargs):
        pass
        
    def callback(self,*args,**kwargs):
        return self._callback(*args,**kwargs)

class SubAction(Action):
    is_terminal = False
    def __init__(self):
        super(SubAction,self).__init__()
        
class TermAction(Action):
    is_terminal = True
    def __init__(self):
        super(TermAction,self).__init__()
        
class CardSelectAction(SubAction):
    def __init__(self, card, selected_container):
        super(CardSelectAction, self).__init__()
        self.text = card.text
        self.card = card
        self.selected_container = selected_container
        
    def _callback(self,*args, **kwargs):
        self.selected_container.append(self.card)
            
        

In [106]:
class Card(object):
    def __init__(self,text):
        self.text = text

In [107]:
class Cards(object):
    pass

In [108]:
class GameException(Exception):
    pass

class PlayerException(Exception):
    pass

In [258]:
class Player(object):
    
    def __init__(self):
        self.mana = 0
        self.hero = None
        self.hero_power = None
        self.deck = []
        self.hand = []
        self.health = 30
        self.armor = 0
        
    def set_deck(self,cards):
        self.deck = cards
        
    def check_ready(self):
        if len(self.deck)<30:
            raise PlayerException("Player has less than 30 cards")
    
    def increase_mana(self):
        if self.mana == 10: return
        self.mana += 1
        
    def draw_card(self):
        new_card = self.deck.pop()
        self.hand.append(new_card)
        
    def first_draw(self,n):
        self.current_draw = [self.deck.pop() for _ in range(n)]
        return self.current_draw
    
    def put_in_hand(self,cards):
        self.hand += cards
        
    def put_in_deck(self,cards):
        self.deck = cards + self.deck
        
    def shuffle_deck(self):
        random.shuffle(self.deck)
        

In [241]:
class Game(object):
    def __init__(self):
        self.p1 = None
        self.p2 = None
        self.turn = 0
        self._current_player = None
        self.started = False
        self.actions = []
        self.execution_thread = Thread(target=self.run)
        self.execution_thread.daemon = True
        
        self.public_actions_buffer = []
        self.is_waiting_for_action = False
        self.selected_action = None
    
    @property
    def current_player(self):
        if not self._current_player:
            self._current_player = self.p1
        return self._current_player
        
    def add_player(self,player):
        player.check_ready()
        if not self.p1:
            self.p1 = player
            return
        if not self.p2:
            self.p2 = player
            return
        raise GameException("There are already 2 players")
        
    def start(self):
        self.execution_thread.start()
        
    def wait_for_action(self):
        self.is_waiting_for_action = True
        self.pause_event = Event()
        self.pause_event.wait()
        self.is_waiting_for_action = False
        return self.selected_action
    
    def resume(self):
        self.pause_event.set()
        
    def _retrieve_actions(self, actions):
        self.public_actions_buffer = ActionsInterfaceList(actions)
        while True:
            action_id = self.wait_for_action()
            action = actions.pop(action_id)
            self.public_actions_buffer = ActionsInterfaceList(actions)
            action.callback()
            if action.is_terminal:
                break
        
    def run(self):
        self.started = True
        self.init_player()
        self.switch_player()
        self.init_player()
        while True:
            self.switch_player()
            self.init_next_turn()
            end = self.play_turn()
            if end:
                break
        
    def init_player(self):
        n_cards = 3 if self.current_player == self.p1 else 4
        gets_coin = self.current_player == self.p2
        draw = self.current_player.first_draw(n_cards)
        selected = []
        actions = [CardSelectAction(card,selected) for card in draw] + [TermAction()]
        self._retrieve_actions(actions)
        print 'remaining actions:'
        print [action.card.text for action in actions]
        print 'selected:'
        print [card.text for card in selected]
        self.current_player.put_in_hand([action.card for action in actions])
        self.current_player.put_in_deck([card for card in selected])
        # self.current_player.shuffle_deck()
        self.current_player.put_in_hand(self.current_player.first_draw(len(selected)))
        
        
    def switch_player(self):
        self._current_player = self.p1 if self.current_player == self.p2 else self.p2
        
        
    def init_next_turn(self):
        self.current_player.increase_mana()
        self.current_player.draw_card()
        
    def play_turn(self):
        
        
    def check_valid_action(self,action_id):
        if action_id in [x[0] for x in self.public_actions_buffer]:
            return True
        raise GameException("Invalid action")
    
    def send_action(self,action_id):
        if not self.is_waiting_for_action:
            raise GameException("No action needed")
        self.check_valid_action(action_id)
        self.selected_action = action_id
        self.resume()
        
    def end_turn(self):
        pass
        
    def get_actions(self):
        if not self.is_waiting_for_action:
            return False
        return self.public_actions_buffer
        

In [242]:
g = Game()

In [243]:
p1 = Player()
p1.set_deck([Card(str(x)) for x in range(30)])

In [244]:
g.add_player(p1)

In [245]:
p2 = Player()
p2.set_deck([Card(str(x+100)) for x in range(30)])

In [246]:
g.add_player(p2)

In [247]:
g.start()

In [248]:
g.get_actions()

[(0, ['29', False]), (1, ['28', False]), (2, ['27', False]), (3, ['', True])]

In [249]:
g.send_action(1)

In [250]:
g.get_actions()

[(0, ['29', False]), (1, ['27', False]), (2, ['', True])]

In [251]:
g.send_action(2)

remaining actions:
['29', '27']
selected:
['28']


In [252]:
g.get_actions()

[(0, ['129', False]),
 (1, ['128', False]),
 (2, ['127', False]),
 (3, ['126', False]),
 (4, ['', True])]

In [253]:
g.send_action(0)

In [254]:
g.get_actions()

[(0, ['128', False]),
 (1, ['127', False]),
 (2, ['126', False]),
 (3, ['', True])]

In [255]:
g.send_action(0)

In [256]:
g.get_actions()

[(0, ['127', False]), (1, ['126', False]), (2, ['', True])]

In [257]:
g.send_action(2)

remaining actions:
['127', '126']
selected:
['129', '128']


In [235]:
g.get_actions()

False

In [236]:
g.is_waiting_for_action

False

In [237]:
p1

In [238]:
p2

In [239]:
[c.text for c in p1.hand]

['29', '27', '26']

In [240]:
[c.text for c in p2.hand]

['127', '126', '125', '124', '123']